In [1]:
import affine, concurrent, cStringIO, glob, IPython, json, os, PIL, random, rasterio, scipy, scipy.ndimage
import shapely, sys, thread, time, traceback, warnings

from collections import defaultdict

#!conda install -y -c conda-forge geopandas
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame

#!conda install -y rasterio
import rasterio
import rasterio.features

%matplotlib inline

In [2]:
class SimpleProcessPoolExecutor(concurrent.futures.ProcessPoolExecutor):
    def __init__(self, max_workers):
        super(SimpleProcessPoolExecutor, self).__init__(max_workers=max_workers)
        self.futures = []
        
    def submit(self, fn, *args, **kwargs):
        future = super(SimpleProcessPoolExecutor, self).submit(fn, *args, **kwargs)
        self.futures.append(future)
        return future
    
    def get_futures(self):
        return self.futures

    def shutdown(self):
        exception_count = 0
        results = []
        for completed in concurrent.futures.as_completed(self.futures):
            try:
                results.append(completed.result())
            except Exception as e:
                exception_count += 1
                sys.stderr.write(
                    'Exception caught in SimpleProcessPoolExecutor.shutdown.  Continuing until all are finished.\n' +
                    'Exception follows:\n' +
                    traceback.format_exc())
        super(SimpleProcessPoolExecutor, self).shutdown()
        if exception_count:
            raise Exception('SimpleProcessPoolExecutor failed: %d of %d raised exception' % (exception_count, len(self.futures)))
        print 'SimpleProcessPoolExecutor succeeded: all %d jobs completed' % (len(self.futures))
        return results

class Stopwatch:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        self.start = time.time()
    def __exit__(self, type, value, traceback):
        sys.stdout.write('%s took %.1f seconds\n' % (self.name, time.time() - self.start))


In [ ]:
warnings.simplefilter('ignore', PIL.Image.DecompressionBombWarning)

def read_src(src):
    ret = scipy.ndimage.imread(src).astype(np.uint16)
    if src.split('/')[0] == 'species_geojsons':
        # original species level were rendered with 255=presence;  scale this down to 1
        ret /= 255
    return ret

def combine_pngs(dest, srcs):
    global z
    total = read_src(srcs[0])
    for src in srcs[1:]:
        total += read_src(src)
    dest_tmp = dest + '.%d.%d.png' % (thread.get_ident(), os.getpid())
    try:
        os.makedirs(os.path.dirname(dest))
    except:
        pass
    # can't use scipy's save since it rescales down to 255 and saves 8-bit
    PIL.Image.fromarray(total).convert('I').save(dest_tmp)
    os.rename(dest_tmp, dest)
    print 'Created %s with max pixel val %d from %d srcs' % (dest, total.max(), len(srcs))
    # Double-check to make sure we don't have weird range and type problems with PNG I/O
    assert np.array_equal(total, read_src(dest))
    
def autoscaled_image(a):
    b = (255.0 / a.max() * a).astype(np.uint8)
    i = PIL.Image.fromarray(b)
    i.thumbnail([1000,1000])
    return i
   
def compute_level(level):
    if level == 4:
        pattern = 'species_geojsons/*.png'
    else:
        pattern = 'level%d/*.png' % (level + 1)
    pngs = sorted(glob.glob(pattern))

    combined = defaultdict(lambda: [])

    for png in pngs:
        combined_name = '_'.join(os.path.splitext(os.path.basename(png))[0].split('_')[0:level])
        if combined_name == '':
            combined_name = 'all'
        combined[combined_name].append(png)
        
    print 'Combining %d pngs at level %d into %d pngs at level %d' % (len(pngs), level + 1, len(combined), level)

    pool = SimpleProcessPoolExecutor(10)

    for c in sorted(combined.keys()):
        dest = 'level%d/%s.png' % (level, c)
        if os.path.exists(dest):
            continue
        pool.submit(combine_pngs, dest, combined[c])
    
    pool.shutdown()

for l in [4,3,2,1,0]:
    compute_level(l)

In [ ]:
autoscaled_image(np.power(scipy.ndimage.imread('level0/all.png').clip(0,255)

In [3]:
a = scipy.ndimage.imread('pre-fish-levels/level0/all.png').clip(0,255).astype(np.uint8)

/home/rsargent/anaconda2/lib/python2.7/site-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (203212800 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [4]:
PIL.Image.fromarray(a).save('biodiv_v2.png')

In [5]:
!identify biodiv_v2.png

biodiv_v2.png PNG 20160x10080 20160x10080+0+0 8-bit RGB 256c 11.87MB 0.000u 0:00.009


In [6]:
!identify -verbose biodiv_v2.png

Image: biodiv_v2.png
  Format: PNG (Portable Network Graphics)
  Mime type: image/png
  Class: PseudoClass
  Geometry: 20160x10080+0+0
  Units: Undefined
  Type: Grayscale
  Endianess: Undefined
  Colorspace: Gray
  Depth: 8-bit
  Channel depth:
    gray: 8-bit
  Channel statistics:
    Pixels: 203212800
    Gray:
      min: 0 (0)
      max: 255 (1)
      mean: 44.4575 (0.174343)
      standard deviation: 53.6818 (0.210517)
      kurtosis: 6.83972
      skewness: 2.60223
  Colors: 256
  Histogram:
  18634506: (  0,  0,  0) #000000 gray(0)
   1161285: (  1,  1,  1) #010101 gray(1)
   8337816: (  2,  2,  2) #020202 gray(2)
   3614637: (  3,  3,  3) #030303 gray(3)
   1377095: (  4,  4,  4) #040404 gray(4)
   1771609: (  5,  5,  5) #050505 gray(5)
   1644733: (  6,  6,  6) #060606 gray(6)
    845060: (  7,  7,  7) #070707 gray(7)
   1042499: (  8,  8,  8) #080808 gray(8)
   1151146: (  9,  9,  9) #090909 gray(9)
   1358395: ( 10, 10, 10) #0A0A0A gray(10)
   1189717: ( 11, 11, 11) #0B0B0B 

In [7]:
!gdal_translate -of Gtiff -a_ullr -180 90 180 -90 -a_srs EPSG:4326 biodiv_v2.png biodiv_v2_4326.tif

Input file size is 20160, 10080
0...10...20...30...40...50...60...70...80...90...100 - done.


In [8]:
!gdalwarp -t_srs EPSG:3857 -ts 32768 32768 -te -20037508.34 -20037508.34 20037508.34 20037508.34 biodiv_v2_4326.tif biodiv_v2_3857.tif

Creating output file that is 32768P x 32768L.
Processing input file biodiv_v2_4326.tif.
0...10...20...30...40...50...60...70...80...90...100 - done.


In [10]:
#!gdal2tiles.py -s EPSG:3857 -z 0-7 merc.tif merc-tiles/
# level 0 = 256 wide (2^0 * 256)
# level 7 = 32768 wide (2^7 * 256)
!gdal2tiles.py -z 0-7 biodiv_v2_3857.tif biodiv_v2

Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100 - done.
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
# rsync -av biodiv_v2 tiles.earthtime.org:/t/earthtime.org/app/data/rsargent-test
# http://tiles.earthtime.org/rsargent-test/biodiv_v2/0/0/0.png

In [ ]:
autoscaled_image(np.power(scipy.ndimage.imread('level0/all.png').clip(0,250) / 250.0, 0.6))

In [ ]:
PIL.Image.fromarray((np.power(scipy.ndimage.imread('level0/all.png').clip(0,255) / 255.0, 0.6) * 255.0).astype(np.uint8))

In [ ]:
!identify level0/all.png

In [ ]:
!ls

In [ ]:
autoscaled_image(np.power(scipy.ndimage.imread('pre-fish-levels/level0/all.png').clip(0,255) / 255.0, 0.5))

In [ ]:
!ls level1

In [ ]:
autoscaled_image(np.power(scipy.ndimage.imread('level1/MAMMALIA.png').clip(0,255) / 255.0, 0.5))

In [ ]:
!gdalwarp -s_srs EPSG:4326 -t_srs EPSG:3857 -te -20037508.34 -20037508.34 20037508.34 20037508.34 pre-fish-levels/level0/all.png  foo.tif

In [ ]:
!gdalwarp --help

In [ ]:
!gdal_translate -of Gtiff -a_ullr -180 90 180 -90 -a_srs EPSG:4326 pre-fish-levels/level0/all.png bla.tif

In [ ]:
!convert -depth 8 pre-fish-levels/level0/all.png biodiv_v2.png

In [ ]:
!identify -verbose biodiv_v2.png

In [ ]:
!gdalinfo bla.tif

In [ ]:
!gdalwarp -t_srs EPSG:3857 -ts 32768 32768 -te -20037508.34 -20037508.34 20037508.34 20037508.34 bla.tif merc.tif

In [ ]:
autoscaled_image(np.power(scipy.ndimage.imread('merc.tif').clip(0,255) / 255.0, 0.5))

In [ ]:
# sudo apt install python-gdal
!gdal2tiles.py -s EPSG:3857 -z 1-6 -a 0 -srcdata=0,0,0 merc.tif merc-tiles/


In [ ]:
!gdal2tiles.py -s EPSG:3857 -z 0-6 merc.tif merc-tiles/


In [ ]:
!which gdalinfo

In [ ]:
ls -lR ~/anaconda2 | grep tiles